In [1]:
from __future__ import print_function, division

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

import time

from dataset import DatasetWrapper
from model import PoseClassModel
from utils.constants import *
from utils.data import get_output_size, eval_class_acc, eval_euc_dists

In [2]:
root = '/data5/ludi/plankton_wi17/pose/poseprediction_torch/records/resnet50/2/'

In [15]:
def read_meta(filename):
    class DotDict(dict):
        __getattr__ = dict.get
        __setattr__ = dict.__setitem__
        __delattr__ = dict.__delitem__
    
    args = DotDict()
    for line in open(meta_file, 'r').read().splitlines():
        if line:
            key, value = line.split(': ')
            try:
                value = int(value)
            except ValueError:
                try:
                    value = float(value)
                except ValueError:
                    pass
            args[key] = value
    return args
        
meta_file = os.path.join(root, 'meta.txt')
args = read_meta(meta_file)
args

{'amp': 1.0,
 'batch_size': 32,
 'data': '/data3/ludi/plankton_wi17/pose/poseprediction_torch/data/',
 'dataset_id': 0,
 'epochs': 50,
 'evaluate': 'False',
 'img_dir': '/data5/Plankton_wi18/rawcolor_db2/images/',
 'input_size': 384,
 'lr': 0.001,
 'lr_step_size': 15,
 'model': 'resnet50',
 'momentum': 0.9,
 'pose_loss_weight': 100.0,
 'resume': '',
 'root': '/data5/ludi/plankton_wi17/pose/poseprediction_torch/records/',
 'start_epoch': 0,
 'std': 3.0,
 'weight_decay': 0.0}

In [4]:
phase = TEST
csv_filename = os.path.join(args.data, 'pose_class/data_{}_{}.csv'.format(phase, args.dataset_id))
num_class = DatasetWrapper.get_num_class(csv_filename.format(phase))
model = PoseClassModel(model_name=args.model, num_class=num_class)
dataset = DatasetWrapper(phase,
                         csv_filename=csv_filename,
                         img_dir=args.img_dir,
                         input_size=(args.input_size, args.input_size),
                         output_size=get_output_size(model, args.input_size),
                         batch_size=16,
                         amp=args.amp,
                         std=args.std)

In [5]:
def loadCheckPoint(model, filename):
    if os.path.isfile(filename):
        print("=> Loading checkpoint '{}'".format(filename))
        checkpoints = torch.load(filename)

        gpuMode = checkpoints.get('gpu_mode', GpuMode.SINGLE)
        if gpuMode == GpuMode.MULTI:
            model = nn.DataParallel(model).cuda()
        elif gpuMode == GpuMode.SINGLE:
            model = model.cuda(0)
        model.load_state_dict(checkpoints['state_dict'])
        print("=> Loaded checkpoint")
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model

checkpoint = os.path.join(root, 'checkpoints/checkpoint-15.pth.tar')
model = loadCheckPoint(model, checkpoint)

=> Loading checkpoint '/data5/ludi/plankton_wi17/pose/poseprediction_torch/records/resnet50/2/checkpoints/checkpoint-15.pth.tar'
=> Loaded checkpoint


In [6]:
class Evaluator(object):
    def __init__(self, dataset, model):
        self.dataset = dataset
        self.model = model
        
        self.mse_loss = nn.MSELoss()
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        
        self.phase = TEST

        cudnn.benchmark = True

        self.log_vars = ['loss', 'class_loss', 'pose_loss', 'class_accuracy', 'pose_error']

#     def to_cuda(self):
#         if self.gpu_mode == GpuMode.MULTI:
#             self.model = nn.DataParallel(self.model).cuda()
#         elif self.gpu_mode == GpuMode.SINGLE:
#             self.model = self.model.cuda(0)
    
    def loadCheckPoint(model, filename):
        if os.path.isfile(filename):
            print("=> Loading checkpoint '{}'".format(filename))
            checkpoints = torch.load(filename)

            gpuMode = checkpoints.get('gpu_mode', GpuMode.SINGLE)
            if gpuMode == GpuMode.MULTI:
                model = nn.DataParallel(model).cuda()
            elif gpuMode == GpuMode.SINGLE:
                model = model.cuda(0)
            model.load_state_dict(checkpoints['state_dict'])
            print("=> Loaded checkpoint")
        else:
            print("=> no checkpoint found at '{}'".format(filename))

        return model

    def to_variable(self, tensor):
#         if self.gpu_mode == GpuMode.MULTI:
#             return Variable(tensor.cuda())
#         elif self.gpu_mode == GpuMode.SINGLE:
        return Variable(tensor.cuda(0))
#         else:
#             return Variable(tensor)

    def evaluate(self):
        self.model.train(False)

        running_vars = {var: 0 for var in self.log_vars}
        total = 0
        epoch_since = time.time()

        for i, data in enumerate(self.dataset.dataloader):
            inputs, target_class, target_map, coordinates = \
                data['image'], data['class_index'], data['target_map'], data['coordinates']
            inputs, target_map, target_class = \
                self.to_variable(inputs), self.to_variable(target_map), self.to_variable(target_class)

            outputs_class, outputs_pose = self.model(inputs)
            loss_class = self.cross_entropy_loss(outputs_class, target_class)
            loss_pose = self.mse_loss(outputs_pose, target_map) * args.pose_loss_weight
            loss = loss_class + loss_pose

            class_acc = eval_class_acc(outputs_class, target_class)
            pose_err = eval_euc_dists(outputs_pose.cpu().data.numpy(), coordinates.numpy())

            vars = {'loss': loss.data[0],
                    'class_loss': loss_class.data[0],
                    'pose_loss': loss_pose.data[0],
                    'class_accuracy': class_acc,
                    'pose_error': pose_err['average']}

            running_vars = {var: running_vars[var] + vars[var] * inputs.size(0) for var in self.log_vars}
            total += inputs.size(0)
            eta = (time.time() - epoch_since) / total * (len(self.dataset) - total)

            term_log = ', '.join(['{}: {:.4f}'.format(var, running_vars[var] / total) for var in self.log_vars])
            print('{} {}/{} ({:.0f}%), {}, ETA: {:.0f}s     \r'
                  .format('Training' if phase == TRAIN else 'validating', total, len(self.dataset),
                          100.0 * total / len(self.dataset), term_log, eta), end='')

        epoch_vars = {var: running_vars[var] / len(self.dataset) for var in self.log_vars}

        print()
        term_log = ', '.join(['{}: {:.4f}'.format(var, epoch_vars[var]) for var in self.log_vars])
        print('{} {} Time Elapsed: {:.0f}s'
              .format(phase, term_log, time.time() - epoch_since))
        print()

In [ ]:
evaluator = Evaluator(dataset=dataset, model=model)
evaluator.evaluate()